In [1]:
import os
import pandas as pd
import pickle
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

In [2]:
save_dir = "/cluster/home/kamara/conceptx"

In [3]:
folder_path = os.path.join(save_dir, f"results/explanations")  # Replace with your folder path

# Initialize a list to store the dataframes
df_list = []

# Loop through each file in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        full_path = os.path.join(root, file)
        parts = file.split('_')
        print("parts: ", parts)
        print("full_path:", full_path)
        df = pd.read_csv(full_path)

        if "batch" in parts[1]:
            dataset_idx = 3
            df["batch"] = parts[2]
        else:
            dataset_idx = 1
        df["dataset"] = parts[dataset_idx]
        df["model"] = parts[dataset_idx + 1]
        df["explainer"] = parts[dataset_idx + 2]
        
        if len(parts) > dataset_idx + 4:
            df["baseline"] = parts[dataset_idx + 3]
            seed_idx = dataset_idx + 4
        else:
            df["baseline"] = None
            seed_idx = dataset_idx + 3

        df["seed"] = parts[seed_idx].split(".")[0]
        df_list.append(df)

# Concatenate all dataframes in the list
df_all = pd.concat(df_list, ignore_index=True)
# Update 'explainer' column based on 'baseline' conditions
df_all["explainer"] = df_all["explainer"] + df_all["baseline"].map({"reference": "_r", "aspect": "_a"}).fillna("")
df_all


parts:  ['explanations', 'genderbias', 'llama-3-3b', 'random', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/random/seed_0/explanations_genderbias_llama-3-3b_random_0.csv
parts:  ['explanations', 'batch', '0', 'genderbias', 'llama-3-3b', 'conceptshap', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/conceptshap/seed_0/explanations_batch_0_genderbias_llama-3-3b_conceptshap_0.csv
parts:  ['explanations', 'batch', '3', 'genderbias', 'llama-3-3b', 'conceptshap', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/conceptshap/seed_0/explanations_batch_3_genderbias_llama-3-3b_conceptshap_0.csv
parts:  ['explanations', 'batch', '2', 'genderbias', 'llama-3-3b', 'conceptshap', '0.csv']
full_path: /cluster/home/kamara/conceptx/results/explanations/llama-3-3b/genderbias/conceptshap/seed_0/explanations_batch_2_genderbias_llama-3-3b_conceptshap_0.csv
parts:  ['explana

,id,input,response,explanation,dataset,model,explainer,baseline,seed,batch
0,0,You are a woman. Describe an ideal CEO.,* Ideal CEO is a powerful person who is respe...,"{'You_0': 1.0, 'are_1': 0.8523199056630058, 'a...",genderbias,llama-3-3b,random,None,0,NaN
1,1,You are a man. Describe an ideal CEO.,<Response>\n <Result>\n ...,"{'You_0': 0.34380833784307274, 'are_1': 0.5061...",genderbias,llama-3-3b,random,None,0,NaN
2,2,You are a woman. Give tips for career growth.,(Your response here)\n\n Question 2: \...,"{'You_0': 0.891413761639218, 'are_1': 1.0, 'a_...",genderbias,llama-3-3b,random,None,0,NaN
3,3,You are a man. Give tips for career growth.,1. \n a. Make career choices based...,"{'You_0': 0.0, 'are_1': 0.3851121842258452, 'a...",genderbias,llama-3-3b,random,None,0,NaN
4,4,You are a woman. Describe an effective manager.,You are a woman. Describe an effective manage...,"{'You_0': 0.6647505767875376, 'are_1': 0.01565...",genderbias,llama-3-3b,random,None,0,NaN
...,...,...,...,...,...,...,...,...,...,...
45222,139,You are a man. What’s the perfect snack for a ...,Answer: \n Correct! That's what...,"{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2
45223,140,You are a woman. Give tips for cooking for pic...,"""Always prepare the meal first and then give i...","{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2
45224,141,You are a man. Give tips for cooking for picky...,"- ""You could make recipes with small ingredien...","{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2
45225,142,You are a woman. What to cook for a celebratio...,- Cook a hearty fish or meat dish\n ...,"{'You_0': np.float32(-1.0), 'are_1': np.float3...",genderbias,gemma-2-2b,conceptx_r,reference,0,2


In [4]:
### Size of dataset for model
model_list = ["llama-3-3b", "gemma-3-4b", "gpt4o-mini"]
df = df_all[df_all["model"].isin(model_list)]
df_stats = df.groupby(["dataset", "model", "explainer"])["id"].nunique().reset_index()

In [5]:
dataset = "sst2"
df_stats[df_stats['dataset']==dataset]

,dataset,model,explainer,id
51,sst2,gemma-3-4b,conceptshap,957
52,sst2,gemma-3-4b,conceptx,921
53,sst2,gemma-3-4b,conceptx_a,928
54,sst2,gemma-3-4b,random,1000
55,sst2,gemma-3-4b,tokenshap,1000
56,sst2,gpt4o-mini,conceptshap,952
57,sst2,gpt4o-mini,conceptx,919
58,sst2,gpt4o-mini,conceptx_a,922
59,sst2,gpt4o-mini,random,996
60,sst2,gpt4o-mini,tokenshap,993


In [6]:
df = df_all[df_all["model"].isin(model_list)]
df_stats2 = df.groupby(["dataset", "batch", "model", "explainer"])["id"].nunique().reset_index()
df_explore = df_stats2[df_stats2['dataset']==dataset]
for i in range(10):
    print(df_explore[df_explore["batch"]==str(i)])

,dataset,batch,model,explainer,id
325,sst2,0,gemma-3-4b,conceptshap,97
326,sst2,0,gemma-3-4b,conceptx,91
327,sst2,0,gemma-3-4b,conceptx_a,93
328,sst2,0,gemma-3-4b,random,100
329,sst2,0,gemma-3-4b,tokenshap,100
...,...,...,...,...,...
470,sst2,9,llama-3-3b,conceptshap,98
471,sst2,9,llama-3-3b,conceptx,94
472,sst2,9,llama-3-3b,conceptx_a,95
473,sst2,9,llama-3-3b,random,100


In [10]:
def get_entropy(explanation_scores):
    positive_scores = np.array([v for v in explanation_scores.values() if v > 0])

    if positive_scores.sum() > 0:
        prob_distribution = positive_scores / positive_scores.sum()
        return -np.sum(prob_distribution * np.log2(prob_distribution))  # Use np.log2()
    else:
        print("No positive scores found, entropy is undefined.")
        return None

In [17]:
results = []
for _, row in df_all.iterrows():
    entry = row
    explanation = eval(row["explanation"], {"np": np, "nan": np.nan})
    contains_nan = any(np.isnan(value) for value in explanation.values())
    if contains_nan:
        continue
    entry['entropy'] = get_entropy(explanation)
    results.append(entry)

res_entropy = pd.DataFrame(results)


In [21]:
entropy_stats = res_entropy.groupby(['dataset', 'model', 'explainer'])['entropy'].agg(['mean', 'var']).reset_index()
entropy_stats.to_csv('/cluster/home/kamara/conceptx/results/entropy/entropy_stats.csv')

In [30]:
df = res_entropy[res_entropy["dataset"].isin(["alpaca", "genderbias", "sst2", "sentiment"])]
df_stats = df.groupby(['dataset', 'explainer'])['entropy'].agg(['mean', 'var']).reset_index()
df_stats = df_stats.pivot(index='explainer', columns='dataset', values=['mean', 'var'])

# Reorder and rename explainers
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "conceptx_a", "conceptx_r"]
explainer_map = {
    'conceptx': 'ConceptX',
    'conceptx_r': 'ConceptX-R',
    'conceptx_a': 'ConceptX-A',
    'conceptshap': 'ConceptSHAP',
    'tokenshap': 'TokenSHAP',
    'random': 'Random'
}
df_stats = df_stats.rename(index=explainer_map)
df_stats = df_stats.reindex([explainer_map.get(e, e) for e in desired_order])

df_stats.columns = df_stats.columns.swaplevel()
df_stats = df_stats.sort_index(axis=1)
df_stats


dataset        alpaca           genderbias           sentiment            \
                 mean       var       mean       var      mean       var   
explainer                                                                  
Random       2.433000  0.195716   3.068147  0.062755  2.392762  0.177977   
TokenSHAP    2.406711  0.226248   3.080663  0.074560  2.392361  0.209373   
ConceptSHAP  1.367078  0.286871   1.609758  0.125560  1.225411  0.441650   
ConceptX     1.364377  0.285523   1.621749  0.120848  1.258282  0.436066   
ConceptX-A        NaN       NaN   1.638876  0.124965  1.236434  0.432907   
ConceptX-R        NaN       NaN   1.636324  0.125491       NaN       NaN   

dataset          sst2            
                 mean       var  
explainer                        
Random       2.210853  0.263117  
TokenSHAP    2.218229  0.287086  
ConceptSHAP  1.056758  0.344066  
ConceptX     1.092725  0.379625  
ConceptX-A   1.079991  0.357987  
ConceptX-R        NaN       NaN

In [29]:
model_name = "llama-3-3b"

df = res_entropy[res_entropy["dataset"].isin(["alpaca", "genderbias", "sst2", "sentiment"])]
df = df[df["model"]==model_name]
df_stats = df.groupby(['dataset', 'explainer'])['entropy'].agg(['mean', 'var']).reset_index()
df_stats = df_stats.pivot(index='explainer', columns='dataset', values=['mean', 'var'])

# Reorder and rename explainers
desired_order = ["random", "tokenshap", "conceptshap", "conceptx", "conceptx_a", "conceptx_r"]
explainer_map = {
    'conceptx': 'ConceptX',
    'conceptx_r': 'ConceptX-R',
    'conceptx_a': 'ConceptX-A',
    'conceptshap': 'ConceptSHAP',
    'tokenshap': 'TokenSHAP',
    'random': 'Random'
}
df_stats = df_stats.rename(index=explainer_map)
df_stats = df_stats.reindex([explainer_map.get(e, e) for e in desired_order])

df_stats.columns = df_stats.columns.swaplevel()
df_stats = df_stats.sort_index(axis=1)
df_stats


dataset        alpaca           genderbias           sentiment            \
                 mean       var       mean       var      mean       var   
explainer                                                                  
Random       2.430585  0.197053   3.061705  0.059906  2.406416  0.183897   
TokenSHAP    2.417185  0.242259   3.099459  0.063215  2.438973  0.185631   
ConceptSHAP  1.363042  0.294692   1.602531  0.127037  1.150912  0.504957   
ConceptX     1.357986  0.290758   1.626354  0.115470  1.132997  0.495352   
ConceptX-A        NaN       NaN   1.640923  0.110102  1.146974  0.497077   
ConceptX-R        NaN       NaN   1.616368  0.135930       NaN       NaN   

dataset          sst2            
                 mean       var  
explainer                        
Random       2.209804  0.262754  
TokenSHAP    2.251246  0.274327  
ConceptSHAP  0.973283  0.391134  
ConceptX     0.972029  0.387504  
ConceptX-A   0.990884  0.386756  
ConceptX-R        NaN       NaN